Link a la documentacion de espressif:  
https://espressif-docs.readthedocs-hosted.com/projects/arduino-esp32/en/latest/api/timer.html#timerattachinterrupt

#Timers con ESP32

Los timers,son contadores ascendentes o descendentes en el tiempo. Son capaces de reiniciarse por si mismos.  
**Puntero**
```
hw_timer_t *nombre;
```
Retorna un puntero a la estructura de tipo timer.

**Configuracion**  
```
nombre = timerBegin(ca=hannel, prescale, mode)
```
- channel: canal del temporizador (0 - 3)  
- prescale: prescala (0 - 65535). La frecuencia de oscilacion del reloj interno, se dividira por la prescala.  
- mode: ascendete (true), descendente (false).  

*Al llamar a timerBegin() el timer iniciara automaticamente el conteo.*



**Funciones de uso**  
```
timerEnd(timer)
timerStart(timer)
timerStop(timer)
timerRestart(timer)
timerRead(timer)
timerReadMicros(timer)
timerReadMillis(timer)
timerReadSeconds(timer) // timepo transcurrido en segundos
timerWrite(timer, val)

// timer --> struct del timer
// val   --> valor del contador(no es el tiempo transcurrido)
```

***Aclaracion:*** Si vamos a usar interrupciones por software, es necesario que la sentencia de configuracion del timer se encuantre antes que la interrupcion.*

## Frecuencia del timer
Frecuencia de oscilacion interna del esp32  
$$ F_{clock}=80MHz $$    
Frecuencia del timer:  
$$
F_{timer}=\frac{F_{clock}}{prescaler}
$$

Suponiendo que tenemos un...  
$prescaler = 50000 = 50 \times 10^{10}$    
$$
F_{timer}=\frac{80000000 Hz}{50000}=1600 Hz
$$

$$
\begin{array}{rcl}
1600 & \leftrightarrow & 1 \, \text{seg} \\
1 & \leftrightarrow & x = \frac{1 \, \text{seg}}{1600} = 0.000625 \, \text{seg} = 625 \, \mu\text{seg}
\end{array}
$$


In [ ]:
def f_Timer(F_clock, prescaler):
  return F_clock/prescaler

def samplesPerSec(f_Timer):
  return 1/f_Timer

def secondsToMicroSeconds(seconds):
  return seconds*1000000

def secondsToNanoSeconds(seconds):
  return seconds*1000000000

In [ ]:
timerFrequency = f_Timer(80000000, 50000)
timerFrequency

1600.0

In [ ]:
sampPerSec = samplesPerSec(timerFrequency)
sampPerSec

0.000625

In [ ]:
samplePerMicroSec = secondsToMicroSeconds(sampPerSec)
samplePerMicroSec

625.0

# Interrupciones por Software

Son segmentos de codigo que se ejecutan cuando sucede un evento interno (por ejemplo que cuando el contador llegue a un cierto valor, dispare la interrupcion)

Se necesita un temporizador para activar la interrupcion  
```
hw_timer_t *nombre = timerBegin(ch, pre, mod);
```
Se requiere un funcion que se ejecutara cuando ocurra la interrupcion  
```
// Funcion que no retorna NINGUN VALOR
void IRAM_ATTR nombre_func()
{
}
```
Las funciones ***IRAM_ATTR*** guardan su valor en la memoria **RAM** y no en la flash como se guardan comunmente. La memoria RAM es mucho mas rapida.  

Se debe **asignar** la funcion con la interrupcion
```
timerAttachInterrupt(timer, handler, edge)
```
- timer: es el nombre del temporizador
- handler: es la funcion que se ejecutara cuando suceda la interrupcion
- edge: puente (true)

# Interrupciones por Hardware

Se ejecuta una seccion de codigo cuando se presenta un evento externo en algun GPIO.

Se debe configurar el GPIO
```
pinMode(pin, mode)
```
Se debe tener una funcion que se active cuando suceda la interrupcion
```
// Funcion que no retorna NINGUN VALOR
void IRAM_ATTR nombre_func()
{
}
```
Se debe **asignar** la funcion de de interrupcion al pin
```
attachInterrupt(pin, handler, mode)
```
- pin: numero de pin del GPIO
- handler: funcion de interrupcion
- mode: modo de operacion

***Modos compatibles de* ```mode```**
- **DISABLED** deshabilitada
- **RISING** flanco de subida (de LOW a HIGH)
- **FALLING** flanco de bajada (de HIGH a LOW)
- **CHANGE** cualquier cambio de estado
- **ONLOW** mientras el estado sea LOW
- **ONHIGH** mientras el estado sea HIGH

***Otras funciones de uso***
```
attachInterruptArg(pin, handler, arg, mode)
```
- arg: argumentos de la funcion

```
detach
```